In [ ]:
### Housekeeping ###
import os

### Data ###
import numpy as np
import pandas as pd
import scipy.io

### Visualization ###
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

### Machine Learning ###
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

### Statistics ###
import scipy.stats as stats
import statsmodels.stats.multitest as smm

### RNA-Seq ###
from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats
from pydeseq2.utils import load_example_data

import gseapy as gp
from gseapy.plot import gseaplot

In [ ]:
dir = os.path.join('..', 'data', 'E-GEOD-60052.csv')
df = pd.read_csv(dir, index_col=0)
df

In [ ]:
df = df.loc[:,~df.columns.duplicated()].copy()
df

In [ ]:
meta_df = df['condition'].reset_index().rename(columns={'index':'sample'})
meta_df = meta_df.set_index('sample').rename_axis(None)
meta_df

In [ ]:
df = df.drop(columns=['condition'])
df

In [ ]:
df.shape

In [ ]:
genes_to_keep = df.columns[df.sum(axis=0) >= 10]
len(genes_to_keep)

In [ ]:
filtered_df = df[genes_to_keep]
filtered_df

In [ ]:
inference = DefaultInference(n_cpus=8)
dds = DeseqDataSet(
    counts=df,
    metadata=meta_df,
    design_factors="condition",
    refit_cooks=True,
    inference=inference,
)

In [ ]:
dds

In [ ]:
dds.deseq2()

In [ ]:
dds

In [ ]:
stat_res = DeseqStats(dds, inference=inference)
stat_res.summary()

In [ ]:
results = stat_res.results_df
results

In [ ]:
results = results[(results['padj'] < 0.05)]
results = results[(abs(results['log2FoldChange']) > 0.05)]
results = results[(results['baseMean'] > 20)]
results

In [ ]:
results.sort_values(by=['log2FoldChange'], ascending=False, inplace=True)
results

In [ ]:
top50 = results.head(50)
bot50 = results.tail(50)
de_res = pd.concat([top50, bot50])
de_res

In [ ]:
dds.layers['normed_counts']

In [ ]:
dds.layers['log1p'] = np.log1p(dds.layers['normed_counts'])
dds.layers['log1p']

In [ ]:
dds_sigs = dds[:, de_res.index]
dds_sigs

In [ ]:
dds_sigs.obs_names

In [ ]:
de_df = pd.DataFrame(dds_sigs.layers['log1p'].T,
                     index=dds_sigs.var_names,
                     columns=dds_sigs.obs_names)
de_df

In [ ]:
sns.clustermap(de_df, z_score=0, cmap='RdBu_r')